In [ ]:
import pandas
import os

# This query represents dataset "Contains Surveys Code" for domain "survey" and was generated for All of Us Registered Tier Dataset v7
# Research All Of Us provided:
#    SELECT
#        answer.person_id,
#        answer.survey_datetime,
#        answer.survey,
#        answer.question_concept_id,
#        answer.question,
#        answer.answer_concept_id,
#        answer.answer,
#        answer.survey_version_concept_id,
#        answer.survey_version_name
dataset_33363199_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer    
    WHERE
        (
            answer.PERSON_ID IN (
                SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (
                        SELECT
                            person_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            person_id IN (
                                SELECT
                                    person_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    value_source_concept_id IN (
                                        SELECT
                                            DISTINCT CAST(c.value AS INT64) as value 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                        JOIN
                                            (
                                                select
                                                    cast(cr.id as string) as id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                WHERE
                                                    concept_id IN (1740639) 
                                                    AND domain_id = 'SURVEY'
                                            ) a 
                                                ON (
                                                    c.path LIKE CONCAT('%.',
                                                a.id,
                                                '.%') 
                                                OR c.path LIKE CONCAT('%.',
                                                a.id) 
                                                OR c.path LIKE CONCAT(a.id,
                                                '.%') 
                                                OR c.path = a.id) 
                                            WHERE
                                                domain_id = 'SURVEY' 
                                                AND type = 'PPI' 
                                                AND subtype = 'ANSWER'
                                            )
                                    )
                    ) 
            )
        )
        AND survey = "Personal and Family Health History"
        AND (question LIKE "%cancer%")
        ORDER BY person_id, question_concept_id
        LIMIT 25
"""

data_frame = pandas.read_gbq(
    dataset_33363199_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

data_frame